In [0]:
import pandas as pd
import joblib
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [0]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter18/Dataset/phpMawTba.csv'

In [0]:
df = pd.read_csv(file_url)

In [6]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [0]:
y = df.pop('class')

In [8]:
cat_columns = [col for col in df.columns if df[col].dtype == 'object']
cat_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.33, random_state=8)

In [0]:
column_categories = {}

In [0]:
for col in cat_columns:
  column_categories[col] = X_train[col].astype('category').cat.categories

In [0]:
pickle.dump(column_categories, open("categories_data.pkl", "wb"))
pickle.dump(cat_columns, open("categorical_columns.pkl", "wb"))

In [0]:
def apply_categories(input_df, cat_dict):
  from pandas.api.types import CategoricalDtype

  for col, cat in cat_dict.items():
    input_df[col] = input_df[col].astype(CategoricalDtype(categories=cat))

  return input_df

In [14]:
X_train_cat = apply_categories(X_train, column_categories)
X_train_cat.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


age                  int64
workclass         category
fnlwgt               int64
education         category
education-num        int64
marital-status    category
occupation        category
relationship      category
race              category
sex               category
capital-gain         int64
capital-loss         int64
hours-per-week       int64
native-country    category
dtype: object

In [0]:
X_train_final = pd.get_dummies(X_train_cat, columns=cat_columns)

In [16]:
rf_model = RandomForestClassifier(random_state=8)
rf_model.fit(X_train_final, y_train)
joblib.dump(rf_model, "model.pkl") 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


['model.pkl']

In [0]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [0]:
app = Flask(__name__)

In [0]:
trained_model = joblib.load("model.pkl")
var_means = pickle.load(open("categories_data.pkl", "rb"))
cat_cols = pickle.load(open("categorical_columns.pkl", "rb"))

In [0]:
@app.route('/api', methods=['POST'])
def predict():
  data = request.get_json()
  df_test = pd.DataFrame(data, index=[0])
  df_test_clean = apply_categories(df_test, var_means)
  df_test_final = pd.get_dummies(df_test_clean, columns=cat_cols)
  prediction = trained_model.predict(df_test_final)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

In [21]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})
flask_thread.start()

 * Serving Flask app "__main__" (lazy loading)


In [22]:
record = X_test.iloc[0,].to_json()
record

'{"age":51,"workclass":" Private","fnlwgt":106151,"education":" 11th","education-num":7,"marital-status":" Divorced","occupation":" Transport-moving","relationship":" Own-child","race":" White","sex":" Male","capital-gain":0,"capital-loss":0,"hours-per-week":40,"native-country":" United-States"}'

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


In [0]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
ip_address = socket.gethostbyname(socket.gethostname())

In [24]:
r = requests.post(f"http://{ip_address}/api", data=record, headers=headers)
r.text

172.28.0.2 - - [25/Nov/2019 03:03:28] "POST /api HTTP/1.1" 200 -


'"[\' <=50K\']"\n'